<a href="https://colab.research.google.com/github/pamsfwang/NLP_word_embedding_space_ML_segmentation/blob/master/extract_bert_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

BERT word embedding extraction

https://towardsdatascience.com/nlp-extract-contextualized-word-embeddings-from-bert-keras-tf-67ef29f60a7b


In [ ]:
!rm -rf bert
!git clone https://github.com/google-research/bert
##add this to resolve error with tf session: module 'tensorflow' has no attribute 'Session'
#!pip3 install --upgrade tensorflow-gpu
import sys
sys.path.append('bert/')
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import codecs
import collections
import json
import re
import os
import pprint
import numpy as np

%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

##modules form BERT
import modeling 
###this includes the BERT model implementation 
import tokenization
###this is for tokenizing the sequences



Cloning into 'bert'...
remote: Enumerating objects: 340, done.
remote: Total 340 (delta 0), reused 0 (delta 0), pack-reused 340
Receiving objects: 100% (340/340), 310.70 KiB | 5.55 MiB/s, done.
Resolving deltas: 100% (186/186), done.
Checking out files: 100% (21/21), done.
1.15.2


In [ ]:
#Mount Google drive on Colab
#step01

!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse


from google.colab import auth
auth.authenticate_user()

from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()

import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
#step02: code for mounting google drive on Linux
!mkdir -p drive
!google-drive-ocamlfuse drive

#step03: change directory to the google drive folder for the project
os.getcwd()
!ls
os.chdir('./drive/memory_strength_construct/colab_scripts/BERT')
os.getcwd()
!ls

E: Package 'python-software-properties' has no installation candidate
··········
 bert
'BERT results.desktop'
 bottomlist_0.35_Noun_fre5_ConcSD5_ConcM5_within_freq50_ConcSD50_ConcM50_trim_crosslists.json
 drive
'extract bert embeddings.ipynb'
 extract_features.ipynb
 extract_features.py
 extract_features_upgrad_report.txt
 input.txt
 toplist_0.4_Noun_fre5_ConcSD5_ConcM5_within_freq50_ConcSD50_ConcM50_trim_crosslists.json
 toplist_bert_sw_to_target.csv
 Untitled0.ipynb
 bert
'BERT results.desktop'
 bottomlist_0.35_Noun_fre5_ConcSD5_ConcM5_within_freq50_ConcSD50_ConcM50_trim_crosslists.json
 drive
'extract bert embeddings.ipynb'
 extract_features.ipynb
 extract_features.py
 extract_features_upgrad_report.txt
 input.txt
 toplist_0.4_Noun_fre5_ConcSD5_ConcM5_within_freq50_ConcSD50_ConcM50_trim_crosslists.json
 toplist_bert_sw_to_target.csv
 Untitled0.ipynb


In [ ]:
#fetch out TPU address from colab 
assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

#A Session object encapsulates the environment in which Operation objects are executed, and Tensor objects are evaluated.
##initialize a new tensorflow session
from google.colab import auth
auth.authenticate_user()
with tf.compat.v1.Session(TPU_ADDRESS) as session:
  ##update tf.Session with tf.compat.v1.Session
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

TPU address is grpc://10.99.112.250:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 3151501292997657875),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 17308523833950466163),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 5866177060730363730),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 8014327793082657448),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 4431956044822057391),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 9841659356441637433),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 12531077520564424514),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 17583582693009141116),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 154710876268

In [ ]:
# Available pretrained model checkpoints:
#   uncased_L-12_H-768_A-12: uncased BERT base model
#   uncased_L-24_H-1024_A-16: uncased BERT large model
#   cased_L-12_H-768_A-12: cased BERT large model
BERT_MODEL = 'uncased_L-24_H-1024_A-16'
BERT_PRETRAINED_DIR = 'gs://cloud-tpu-checkpoints/bert/' + BERT_MODEL
print('***** BERT pretrained directory: {} *****'.format(BERT_PRETRAINED_DIR))
!gsutil ls $BERT_PRETRAINED_DIR

***** BERT pretrained directory: gs://cloud-tpu-checkpoints/bert/uncased_L-24_H-1024_A-16 *****
gs://cloud-tpu-checkpoints/bert/uncased_L-24_H-1024_A-16/bert_config.json
gs://cloud-tpu-checkpoints/bert/uncased_L-24_H-1024_A-16/bert_model.ckpt.data-00000-of-00001
gs://cloud-tpu-checkpoints/bert/uncased_L-24_H-1024_A-16/bert_model.ckpt.index
gs://cloud-tpu-checkpoints/bert/uncased_L-24_H-1024_A-16/bert_model.ckpt.meta
gs://cloud-tpu-checkpoints/bert/uncased_L-24_H-1024_A-16/checkpoint
gs://cloud-tpu-checkpoints/bert/uncased_L-24_H-1024_A-16/vocab.txt


* A TensorFlow checkpoint (bert_model.ckpt) containing the pre-trained 
weights (which is actually 3 files).
* A vocab file (vocab.txt) to map WordPiece to word id.
* A config file (bert_config.json) which specifies the hyperparameters of the model.

In [ ]:
LAYERS = [-1,-2,-3,-4]
#we will use the last 4 years
NUM_TPU_CORES = 8
MAX_SEQ_LENGTH = 128
BERT_CONFIG = BERT_PRETRAINED_DIR + '/bert_config.json'
CHKPT_DIR = BERT_PRETRAINED_DIR + '/bert_model.ckpt'
VOCAB_FILE = BERT_PRETRAINED_DIR + '/vocab.txt'
INIT_CHECKPOINT = BERT_PRETRAINED_DIR + '/bert_model.ckpt'
BATCH_SIZE = 128

In [ ]:
class InputExample(object):

  def __init__(self, unique_id, text_a, text_b=None):
    self.unique_id = unique_id
    self.text_a = text_a
    self.text_b = text_b
    ##text_b = None: we aim to use single sequences ranther than a sequence pair

In [ ]:
#this is what BERT for input
class InputFeatures(object):
  """A single set of features of data."""

  def __init__(self, unique_id, tokens, input_ids, input_mask, input_type_ids):
    self.unique_id = unique_id
    self.tokens = tokens
    #a vector of input tokens
    self.input_ids = input_ids
    #ids that correspond to the token from the vocabulary
    self.input_mask = input_mask
    #real token sequence from padding
    self.input_type_ids = input_type_ids
    #separate segment A from segment B

In [ ]:
def input_fn_builder(features, seq_length):
  """Creates an `input_fn` closure to be passed to TPUEstimator."""

  all_unique_ids = []
  all_input_ids = []
  all_input_mask = []
  all_input_type_ids = []

  for feature in features:
    all_unique_ids.append(feature.unique_id)
    all_input_ids.append(feature.input_ids)
    all_input_mask.append(feature.input_mask)
    all_input_type_ids.append(feature.input_type_ids)

  def input_fn(params):
    """The actual input function."""
    batch_size = params["batch_size"]

    num_examples = len(features)

    # This is for demo purposes and does NOT scale to large data sets. We do
    # not use Dataset.from_generator() because that uses tf.py_func which is
    # not TPU compatible. The right way to load data is with TFRecordReader.
    d = tf.data.Dataset.from_tensor_slices({
        "unique_ids":
            tf.constant(all_unique_ids, shape=[num_examples], dtype=tf.int32),
        "input_ids":
            tf.constant(
                all_input_ids, shape=[num_examples, seq_length],
                dtype=tf.int32),
        "input_mask":
            tf.constant(
                all_input_mask,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "input_type_ids":
            tf.constant(
                all_input_type_ids,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
    })

    d = d.batch(batch_size=batch_size, drop_remainder=False)
    return d

  return input_fn
  
def model_fn_builder(bert_config, init_checkpoint, layer_indexes, use_tpu,
                     use_one_hot_embeddings):
  """Returns `model_fn` closure for TPUEstimator."""

  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    unique_ids = features["unique_ids"]
    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    input_type_ids = features["input_type_ids"]
 ##define the model
    model = modeling.BertModel(
        config=bert_config,
        is_training=False,
        input_ids=input_ids,
        input_mask=input_mask,
        token_type_ids=input_type_ids,
        use_one_hot_embeddings=use_one_hot_embeddings)

    if mode != tf.estimator.ModeKeys.PREDICT:
      raise ValueError("Only PREDICT modes are supported: %s" % (mode))

    tvars = tf.trainable_variables()
    scaffold_fn = None
    (assignment_map,
     initialized_variable_names) = modeling.get_assignment_map_from_checkpoint(
         tvars, init_checkpoint)
    if use_tpu:

      def tpu_scaffold():
        tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
        return tf.train.Scaffold()

      scaffold_fn = tpu_scaffold
    else:
      tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

    tf.logging.info("**** Trainable Variables ****")
    for var in tvars:
      init_string = ""
      if var.name in initialized_variable_names:
        init_string = ", *INIT_FROM_CKPT*"
      tf.logging.info("  name = %s, shape = %s%s", var.name, var.shape,
                      init_string)

    all_layers = model.get_all_encoder_layers()

    predictions = {
        "unique_id": unique_ids,
    }

## extract data from the last 4 layers
    for (i, layer_index) in enumerate(layer_indexes):
      predictions["layer_output_%d" % i] = all_layers[layer_index]

    output_spec = tf.contrib.tpu.TPUEstimatorSpec(
        mode=mode, predictions=predictions, scaffold_fn=scaffold_fn)
    return output_spec

  return model_fn

In [ ]:
def convert_examples_to_features(examples, seq_length, tokenizer):
  """Loads a data file into a list of `InputBatch`s."""

  features = []
  for (ex_index, example) in enumerate(examples):
    tokens_a = tokenizer.tokenize(example.text_a)

    tokens_b = None
    if example.text_b:
      tokens_b = tokenizer.tokenize(example.text_b)

    if tokens_b:
      # Modifies `tokens_a` and `tokens_b` in place so that the total
      # length is less than the specified length.
      # Account for [CLS], [SEP], [SEP] with "- 3"
      _truncate_seq_pair(tokens_a, tokens_b, seq_length - 3)
    else:
      # Account for [CLS] and [SEP] with "- 2"
      if len(tokens_a) > seq_length - 2:
        tokens_a = tokens_a[0:(seq_length - 2)]

    # The convention in BERT is:
    # (a) For sequence pairs:
    #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
    #  type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
    # (b) For single sequences:
    #  tokens:   [CLS] the dog is hairy . [SEP]
    #  type_ids: 0     0   0   0  0     0 0
    #
    # Where "type_ids" are used to indicate whether this is the first
    # sequence or the second sequence. The embedding vectors for `type=0` and
    # `type=1` were learned during pre-training and are added to the wordpiece
    # embedding vector (and position vector). This is not *strictly* necessary
    # since the [SEP] token unambiguously separates the sequences, but it makes
    # it easier for the model to learn the concept of sequences.
    #
    # For classification tasks, the first vector (corresponding to [CLS]) is
    # used as as the "sentence vector". Note that this only makes sense because
    # the entire model is fine-tuned.
    tokens = []
    input_type_ids = []
    tokens.append("[CLS]")
    input_type_ids.append(0)
    for token in tokens_a:
      tokens.append(token)
      input_type_ids.append(0)
    tokens.append("[SEP]")
    input_type_ids.append(0)

    if tokens_b:
      for token in tokens_b:
        tokens.append(token)
        input_type_ids.append(1)
      tokens.append("[SEP]")
      input_type_ids.append(1)

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [1] * len(input_ids)

    # Zero-pad up to the sequence length.
    while len(input_ids) < seq_length:
      input_ids.append(0)
      input_mask.append(0)
      input_type_ids.append(0)

    assert len(input_ids) == seq_length
    assert len(input_mask) == seq_length
    assert len(input_type_ids) == seq_length

    if ex_index < 5:
      tf.logging.info("*** Example ***")
      tf.logging.info("unique_id: %s" % (example.unique_id))
      tf.logging.info("tokens: %s" % " ".join(
          [tokenization.printable_text(x) for x in tokens]))
      tf.logging.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
      tf.logging.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
      tf.logging.info(
          "input_type_ids: %s" % " ".join([str(x) for x in input_type_ids]))

    features.append(
        InputFeatures(
            unique_id=example.unique_id,
            tokens=tokens,
            input_ids=input_ids,
            input_mask=input_mask,
            input_type_ids=input_type_ids))
  return features

def _truncate_seq_pair(tokens_a, tokens_b, max_length):
  """Truncates a sequence pair in place to the maximum length."""

  # This is a simple heuristic which will always truncate the longer sequence
  # one token at a time. This makes more sense than truncating an equal percent
  # of tokens from each, since if one sequence is very short then each token
  # that's truncated likely contains more information than a longer sequence.
  while True:
    total_length = len(tokens_a) + len(tokens_b)
    if total_length <= max_length:
      break
    if len(tokens_a) > len(tokens_b):
      tokens_a.pop()
    else:
      tokens_b.pop()

In [ ]:
def read_sequence(input_sentences):
  examples = []
  unique_id = 0
  for sentence in input_sentences:
    line = tokenization.convert_to_unicode(sentence)
    examples.append(InputExample(unique_id=unique_id, text_a=line))
    unique_id += 1
  return examples
  

In [ ]:
def get_features(input_text, dim=1024):
  layer_indexes = LAYERS

  bert_config = modeling.BertConfig.from_json_file(BERT_CONFIG)

  tokenizer = tokenization.FullTokenizer(
      vocab_file=VOCAB_FILE, do_lower_case=True)

  is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2
  tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)
  run_config = tf.contrib.tpu.RunConfig(
      cluster=tpu_cluster_resolver,
      tpu_config=tf.contrib.tpu.TPUConfig(
          num_shards=NUM_TPU_CORES,
          per_host_input_for_training=is_per_host))

  examples = read_sequence(input_text)

  features = convert_examples_to_features(
      examples=examples, seq_length=MAX_SEQ_LENGTH, tokenizer=tokenizer)
  print(features)

  unique_id_to_feature = {}
  for feature in features:
    unique_id_to_feature[feature.unique_id] = feature

  model_fn = model_fn_builder(
      bert_config=bert_config,
      init_checkpoint=INIT_CHECKPOINT,
      layer_indexes=layer_indexes,
      use_tpu=True,
      use_one_hot_embeddings=True)

  # If TPU is not available, this will fall back to normal Estimator on CPU
  # or GPU.
  estimator = tf.contrib.tpu.TPUEstimator(
      use_tpu=True,
      model_fn=model_fn,
      config=run_config,
      predict_batch_size=BATCH_SIZE,
      train_batch_size=BATCH_SIZE)

  input_fn = input_fn_builder(
      features=features, seq_length=MAX_SEQ_LENGTH)

  # Get features
  for result in estimator.predict(input_fn, yield_single_examples=True):
    unique_id = int(result["unique_id"])
    feature = unique_id_to_feature[unique_id]
    output = collections.OrderedDict()
    for (i, token) in enumerate(feature.tokens):
      layers = []
      for (j, layer_index) in enumerate(layer_indexes):
        layer_output = result["layer_output_%d" % j]
        layer_output_flat = np.array([x for x in layer_output[i:(i + 1)].flat])
        layers.append(layer_output_flat)
      output[token] = sum(layers)[:dim]
  
  return output

In [ ]:
embeddings = get_features(['test is not a sentence'])


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json, os, pandas, pickle
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
high_embeddings = get_features(['date month deadline beginning year birth'])
low_embeddings = get_features(['date availability countdown periods billing booking'])
random_low_embeddings = get_features(['availability countdown date periods billing booking'])
mised_high_low = get_features(['date month deadline beginning year birth availability countdown periods billing booking'])

position_embeddings = get_features(['date month'])
position_embeddings2 = get_features(['month date'])

In [ ]:
high_embeddings = get_features(['date month deadline beginning year birth'])
low_embeddings = get_features(['date availability countdown periods billing booking'])
random_low_embeddings = get_features(['availability countdown date periods billing booking'])
mised_high_low = get_features(['date month deadline beginning year birth availability countdown periods billing booking'])

position_embeddings = get_features(['date month'])
position_embeddings2 = get_features(['month date'])

In [ ]:
items = list(embeddings.items())
type(items[1][1])

vec = []
vec.append(items[1][1])
vec.append(items[2][1])

len(vec)
cosine_similarity(vec)

In [ ]:
#position_embeddings = get_features(['date month'])
position_embeddings = get_features(['date month'])
position_embeddings = list(position_embeddings.items())
vec = []
vec.append(position_embeddings[1][1])
vec.append(position_embeddings[2][1])
dm_cs = cosine_similarity(vec)

position_embeddings2 = get_features(['month date'])
position_embeddings2 = list(position_embeddings2.items())
vec = []
vec.append(position_embeddings2[1][1])
vec.append(position_embeddings2[2][1])

md_cs = cosine_similarity(vec)


In [ ]:
print("date month: "+str(dm_cs[0,1])+" month date:"+str(md_cs[0,1]))

date month: 0.5650047 month date:0.59286714


In [ ]:
#high_embeddings = list(high_embeddings.items())
highcs=[]
for i,token in enumerate(high_embeddings):
  target_emb = high_embeddings[1][1]

  if (i>0) & (i!=7):
    print(high_embeddings[i][0])
    vec = []
    vec.append(target_emb)
    vec.append(high_embeddings[i][1])
    highcs.append(cosine_similarity(vec)[0,1])

highcs

date
month
deadline
beginning
year
birth


[1.0000005, 0.59399307, 0.7992438, 0.6398051, 0.6897364, 0.5747025]

In [ ]:
#low_embeddings = list(low_embeddings.items())
lowcs=[]
for i,token in enumerate(low_embeddings):
  target_emb = low_embeddings[1][1]

  if (i>0) & (i!=7):
    print(low_embeddings[i][0])
    vec = []
    vec.append(target_emb)
    vec.append(low_embeddings[i][1])
    lowcs.append(cosine_similarity(vec)[0,1])

lowcs

date
availability
countdown
periods
billing
booking


[0.99999994, 0.7389654, 0.7088387, 0.73258114, 0.7085004, 0.59022087]

In [ ]:
#mised_high_low =list(mised_high_low.items())
mixcs = []
for i,token in enumerate(mised_high_low):
  target_emb = mised_high_low[1][1]

  if (i>0) & (i!=13):
    print(mised_high_low[i][0])
    vec = []
    vec.append(target_emb)
    vec.append(mised_high_low[i][1])
    mixcs.append(cosine_similarity(vec)[0,1])

mixcs

date
month
deadline
beginning
year
birth
availability
countdown
periods
billing
booking
[SEP]


[1.0,
 0.63515663,
 0.75330454,
 0.5994988,
 0.67003405,
 0.564377,
 0.61225647,
 0.6399303,
 0.6119607,
 0.5453922,
 0.5296505,
 0.16584453]

In [ ]:
#'availability countdown date periods billing booking'
#random_low_embeddings = list(random_low_embeddings.items())
midcs = []
for i,token in enumerate(random_low_embeddings):
  target_emb = random_low_embeddings[3][1]

  if (i>0) & (i!=7):
    print(random_low_embeddings[i][0])
    vec = []
    vec.append(target_emb)
    vec.append(random_low_embeddings[i][1])
    midcs.append(cosine_similarity(vec)[0,1])

midcs

availability
countdown
date
periods
billing
booking


[0.6546845, 0.7327888, 0.99999976, 0.7723243, 0.68812525, 0.62147033]

In [ ]:
random_low_embeddings = list(random_low_embeddings.items())
midcs = []
for i,token in enumerate(random_low_embeddings):
  print(i,token)

0 ('[CLS]', array([-0.5416696 , -0.9207057 , -1.5444452 , ..., -0.59676135,
       -1.4708406 ,  1.6912278 ], dtype=float32))
1 ('availability', array([-1.3498415 , -0.9689705 , -0.94484186, ...,  0.9402225 ,
        0.94068754,  1.0125138 ], dtype=float32))
2 ('countdown', array([ 0.54941446, -1.0694321 , -1.5869894 , ..., -2.0208044 ,
       -1.1741501 , -2.0848913 ], dtype=float32))
3 ('date', array([ 0.96290064, -4.081417  ,  0.50267255, ...,  0.7004739 ,
       -0.94479287,  0.38038516], dtype=float32))
4 ('periods', array([ 1.3098817 , -1.7351489 ,  0.14386044, ...,  0.20408109,
        0.41994444, -0.9173153 ], dtype=float32))
5 ('billing', array([-0.2296798 , -2.8875034 , -0.77522445, ...,  2.2388723 ,
        0.30182323,  1.8274653 ], dtype=float32))
6 ('booking', array([-2.4484565, -0.9027653, -3.1282434, ...,  2.7456594, -1.2319993,
       -0.0944172], dtype=float32))
7 ('[SEP]', array([ 0.16669288,  0.74672574,  0.25964472, ..., -0.52716374,
       -1.3481613 ,  0.58092874]

In [ ]:
high_embeddings = list(high_embeddings.items())

In [ ]:
len(high_embeddings)

8

In [ ]:
position_embeddings = list(position_embeddings.items())

for i, token in enumerate(items):
  target_emb = items[1][1]

  print(i)
  print(token)

len(items)

0
('[CLS]', array([-0.16518201, -4.0077515 , -3.246676  , ..., -1.2020907 ,
       -3.3798065 ,  2.6246133 ], dtype=float32))
1
('test', array([ 0.5504674, -2.3299797,  1.8608823, ...,  3.422434 , -1.9182506,
       -0.8740864], dtype=float32))
2
('is', array([ 1.311218  , -3.4386044 , -1.9155076 , ...,  1.5354011 ,
       -0.5726017 ,  0.28953427], dtype=float32))
3
('not', array([ 0.19106065, -2.027442  ,  0.8313433 , ...,  3.2049594 ,
        1.4314429 ,  2.7884793 ], dtype=float32))
4
('a', array([-1.1081269 ,  1.0753058 ,  1.1767533 , ..., -0.45109302,
       -2.1284971 ,  0.6042762 ], dtype=float32))
5
('sentence', array([ 1.9701583, -3.4249117, -3.3920226, ...,  1.3111655, -2.7660823,
        3.6116614], dtype=float32))
6
('[SEP]', array([-0.77259606, -0.596901  , -0.2573238 , ...,  0.30207834,
       -1.3111746 , -0.8363816 ], dtype=float32))


7

Training without target word

In [ ]:
high_embeddings = get_features(['month deadline beginning year birth'])
low_embeddings = get_features(['availability countdown periods billing booking'])
mised_high_low = get_features(['month deadline beginning year birth availability countdown periods billing booking'])

position_embeddings = get_features(['month'])
#target_embeddings = get_features(['date'])

In [ ]:
target_embeddings = get_features(['date'])
target_embeddings = list(target_embeddings.items())
target_embeddings

In [ ]:
date_emd = target_embeddings[1][1]

In [ ]:
#high_embeddings = list(high_embeddings.items())
highcs = []
for i,token in enumerate(high_embeddings):
  if (i>0) & (i!=6):
    print(high_embeddings[i][0])
    vec = []
    vec.append(date_emd)
    vec.append(high_embeddings[i][1])
    highcs.append(cosine_similarity(vec)[0,1])

highcs

month
deadline
beginning
year
birth


[0.6309255, 0.659688, 0.57426834, 0.6630543, 0.59513235]

In [ ]:
low_embeddings = list(low_embeddings.items())
lowcs = []
for i,token in enumerate(low_embeddings):
  if (i>0) & (i!=6):
    print(low_embeddings[i][0])
    vec = []
    vec.append(date_emd)
    vec.append(low_embeddings[i][1])
    lowcs.append(cosine_similarity(vec)[0,1])

lowcs

availability
countdown
periods
billing
booking


[0.50866324, 0.5768784, 0.6088849, 0.50817883, 0.5647483]

In [ ]:
#mised_high_embeddings = list(mised_high_low.items())
mised_highcs = []
for i,token in enumerate(mised_high_embeddings):
  if (i>0) & (i!=11):
    print(mised_high_embeddings[i][0])
    vec = []
    vec.append(date_emd)
    vec.append(mised_high_embeddings[i][1])
    mised_highcs.append(cosine_similarity(vec)[0,1])

mised_highcs

month
deadline
beginning
year
birth
availability
countdown
periods
billing
booking


[0.59246194,
 0.57624924,
 0.49211362,
 0.62165123,
 0.48195985,
 0.49016652,
 0.5085995,
 0.565302,
 0.47163138,
 0.5118553]

In [ ]:
month_emb = list(position_embeddings.items())
vec = []
vec.append(date_emd)
vec.append(month_emb[1][1])
cosine_similarity(vec)[0,1]

0.6314087

In [ ]:
month_emb[1][0]

'month'

In [ ]:
random_low_embeddings = get_features(['availability periods billing countdown booking'])

In [ ]:
random_embeddings = list(random_low_embeddings.items())
random_lowcs = []
for i,token in enumerate(random_embeddings):
  if (i>0) & (i!=6):
    print(random_embeddings[i][0])
    vec = []
    vec.append(date_emd)
    vec.append(random_embeddings[i][1])
    random_lowcs.append(cosine_similarity(vec)[0,1])

random_lowcs

availability
periods
billing
countdown
booking


[0.4941221, 0.62298566, 0.5350395, 0.5854438, 0.5730567]

Calculate bert similarity for lists


In [ ]:
wordfname = 'bottomlist_0.35_Noun_fre5_ConcSD5_ConcM5_within_freq50_ConcSD50_ConcM50_trim_crosslists.json'
#'toplist_0.4_Noun_fre5_ConcSD5_ConcM5_within_freq50_ConcSD50_ConcM50_trim_crosslists.json'
with open(wordfname) as json_file:
    common_words = json.load(json_file)


In [ ]:
common_words['L001']

['date', 'availability', 'countdown', 'periods', 'billing', 'booking']

In [ ]:
modeled_df = pd.DataFrame()
for ii, ikey in enumerate(common_words):
  #target word emb
  target_word = get_features([common_words[ikey][0]])
  target_word_emb = list(target_word.items())
  target_word_emb = target_word_emb[1][1]

  #studied word emb
  sw = ' '.join(common_words[ikey][1:])
  sw_emb = get_features([sw])
  sw_emb = list(sw_emb.items())

  sw_target_cs= []
  for isw in range(1,6):
    print(sw_emb[isw][0])
    vec = []
    vec.append(target_word_emb)
    vec.append(sw_emb[isw][1])
    sw_target_cs.append(cosine_similarity(vec)[0,1])

  modeled_lists = {}
  modeled_lists['list_membership']={}
  modeled_lists['list_target']={}
  modeled_lists['bert_cs']={}
  modeled_lists['studied_words']={}

  modeled_lists['list_membership'] = [ikey]*len(sw_target_cs)
  modeled_lists['list_target'] = [common_words[ikey][0]]*len(sw_target_cs)
  modeled_lists['bert_cs'] = sw_target_cs
  modeled_lists['studied_words'] = sw.split(' ')

  temp_df = pd.DataFrame.from_dict(modeled_lists)
  modeled_df = modeled_df.append(temp_df,ignore_index=True)
  modeled_df.to_csv('bottomlist_bert_sw_to_target.csv')


Streaming output truncated to the last 5000 lines.
INFO:tensorflow:  name = bert/encoder/layer_6/attention/output/LayerNorm/gamma:0, shape = (1024,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_6/intermediate/dense/kernel:0, shape = (1024, 4096), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_6/intermediate/dense/bias:0, shape = (4096,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_6/output/dense/kernel:0, shape = (4096, 1024), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_6/output/dense/bias:0, shape = (1024,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_6/output/LayerNorm/beta:0, shape = (1024,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_6/output/LayerNorm/gamma:0, shape = (1024,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_7/attention/self/query/kernel:0, shape = (1024, 1024), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_7/attention/self/query/bias:0, s

In [ ]:
modeled_df.head()

,list_membership,list_target,bert_cs,studied_words
0,L001,date,0.630925,month
1,L001,date,0.659688,deadline
2,L001,date,0.574268,beginning
3,L001,date,0.663054,year
4,L001,date,0.595132,birth


In [ ]:
modeled_df.to_csv('toplist_bert_sw_to_target.csv')